## Inpainting with Variable Dataset Size

Recall that the Image网 dataset consists of:

1. A `/val` folder with 10 classes.
2. A `/train` folder with 20 classes. 
  - There are ~125 images in each class that exists in `/val`. There are 
  - There are ~1,300 images in each class that does not exist in `/val`
3. An `/unsup` folder with 7,750 unlabelled images.

The question we would like to answer with this notebook is:

> What is the effect of dataset size during pretext training on downstream task performance?

To answer this question we will consider four different datasets, each built from ImageWang.

They are:

1. All data in `/train`, `/unsup` and `/val`
2. All data in `/train`, `/unsup`
3. All data in `/train`
4. Only Data in `/train` that has a corresponding class in `/val`

In [1]:
import json
import torch

import numpy as np

from functools import partial


from fastai2.layers import MishJit, MaxPool, LabelSmoothingCrossEntropy
from fastai2.basics import DataBlock, RandomSplitter, GrandparentSplitter
from fastai2.learner import Learner
from fastai2.metrics import accuracy, top_k_accuracy
from fastai2.optimizer import ranger, Adam, SGD, RMSProp

from fastai2.vision.all import ImageBlock, PILMask, get_image_files, PILImage, imagenet_stats
from fastai2.vision.core import get_annotations, Image, TensorBBox, TensorPoint, TensorImage
from fastai2.vision.augment import aug_transforms, RandomResizedCrop, RandTransform, FlipItem
from fastai2.vision.learner import unet_learner, unet_config
from fastai2.vision.models.xresnet import xresnet50, xresnet34

from fastai2.data.transforms import get_files
from fastai2.data.external import download_url, URLs, untar_data

from fastcore.foundation import L
from fastcore.utils import num_cpus

from torch.nn import MSELoss
from torchvision.models import resnet34

We will train this network with the best hyper-parameters/optimizer/settings we know. These settings come from [training Imagenette](https://github.com/fastai/imagenette/tree/58a63175a2c6457650289d32741940d6a7d58fbf). 

One thing to keep in mind is that the above is a classification task, so it's not 100% guaranteed that these settings will map perfectly to our task. That said, they're probably a very good starting point.

As of January 2020 the [best parameters](https://github.com/fastai/imagenette/blob/58a63175a2c6457650289d32741940d6a7d58fbf/2020-01-train.md) are:

```
--lr 8e-3 
--sqrmom 0.99 
--mom 0.95 
--eps 1e-6 
--bs 64 
--opt ranger 
--sa 1
--fp16 1 
--arch xse_resnext50 
--pool MaxPool
```

One change we're making is that we're going to use **`xresnet34`** here. 

In [2]:
# Default parameters
gpu=None
lr=1e-2
size=128
sqrmom=0.99
mom=0.9
eps=1e-6
epochs=15
bs=64
mixup=0.
opt='ranger',
arch='xresnet50'
sh=0.
sa=0
sym=0
beta=0.
act_fn='MishJit'
fp16=0
pool='AvgPool',
dump=0
runs=1
meta=''

# Chosen parameters
lr=8e-3
sqrmom=0.99
mom=0.95
eps=1e-6
bs=64 
opt='ranger'
sa=1                 #NOTE: NOT USED HERE. Do we need this?
fp16=0               #NOTE: My GPU cannot run fp16 :'(
arch='xresnet50' 
pool='MaxPool'

gpu=0

# NOTE: Normally loaded from their corresponding string
m = xresnet34
act_fn = MishJit
pool = MaxPool

In [3]:
# We create this dummy class in order to create a transform that ONLY operates on images of this type
# We will use it to create all input images
class PILImageInput(PILImage): pass

class RandomCutout(RandTransform):
    "Picks a random scaled crop of an image and resize it to `size`"
    split_idx = None
    def __init__(self, min_n_holes=5, max_n_holes=10, min_length=5, max_length=50, **kwargs):
        super().__init__(**kwargs)
        self.min_n_holes=min_n_holes
        self.max_n_holes=max_n_holes
        self.min_length=min_length
        self.max_length=max_length
        

    def encodes(self, x:PILImageInput):
        """
        Note that we're accepting our dummy PILImageInput class
        fastai2 will only pass images of this type to our encoder. 
        This means that our transform will only be applied to input images and won't
        be run against output images.
        """
        
        n_holes = np.random.randint(self.min_n_holes, self.max_n_holes)
        pixels = np.array(x) # Convert to mutable numpy array. FeelsBadMan
        h,w = pixels.shape[:2]

        for n in range(n_holes):
            h_length = np.random.randint(self.min_length, self.max_length)
            w_length = np.random.randint(self.min_length, self.max_length)
            h_y = np.random.randint(0, h)
            h_x = np.random.randint(0, w)
            y1 = int(np.clip(h_y - h_length / 2, 0, h))
            y2 = int(np.clip(h_y + h_length / 2, 0, h))
            x1 = int(np.clip(h_x - w_length / 2, 0, w))
            x2 = int(np.clip(h_x + w_length / 2, 0, w))
           
            pixels[y1:y2, x1:x2, :] = 0
            
        return Image.fromarray(pixels, mode='RGB')

In [4]:
source = untar_data(URLs.IMAGEWANG_160)

In [5]:
opt_func = partial(ranger, mom=mom, sqr_mom=sqrmom, eps=eps, beta=beta)

In [6]:
# Number of workers for creating the data bunch
workers = min(8, num_cpus())

In [7]:
# transforms are the same for each experiment
item_tfms=[RandomResizedCrop(size, min_scale=0.35), FlipItem(0.5), RandomCutout]
batch_tfms=RandomErasing(p=0.9, max_count=3, sh=sh) if sh else None

In [8]:
size = 160
#CHANGE: I can only fit ~32 images in a batch
bs = 32

## Get Items From Folder

So before we do anything, let's create some helper methods that will give us only the training sets that we would like.

In [9]:
def get_all_items(path):
    return get_files(path, extensions='.JPEG', recurse=True)

def get_train_items(path):
    return get_files(path/'train', extensions='.JPEG', recurse=True)

def get_unsup_items(path):
    return get_files(path/'unsup', extensions='.JPEG', recurse=True)

def get_valid_items(path):
    return get_files(path/'val', extensions='.JPEG', recurse=True)

def get_train_and_unsup(path):
    return get_train_items(path) + get_unsup_items(path)

def get_train_items_that_are_present_in_val(path):
    """
    We first get a list of all classes in /val
    Then we use that list to get all the examples of each class from /train
    """
    val = source/'val'
    validation_classes = [path.name for path in val.iterdir()]
    
    train_files = L()
    for class_name in validation_classes:
        items = get_files(path/'train'/class_name, extensions='.JPEG', recurse=True)
        train_files = train_files + items
        
    return train_files

all_items = get_all_items(untar_data(URLs.IMAGEWANG_160))
train_items = get_train_items(untar_data(URLs.IMAGEWANG_160))
unsup_items = get_unsup_items(untar_data(URLs.IMAGEWANG_160))
valid_items = get_valid_items(untar_data(URLs.IMAGEWANG_160))

print("All Files: {}".format(len(all_items)))
print("Train Files: {}".format(len(train_items)))
print("Unsup Files: {}".format(len(unsup_items)))
print("Valid Files: {}".format(len(valid_items)))
print()

train_and_unsup_items = get_train_and_unsup(untar_data(URLs.IMAGEWANG_160))
print("Train+Unsup Files: {}".format(len(train_and_unsup_items)))
train_in_valid_items = get_train_items_that_are_present_in_val(untar_data(URLs.IMAGEWANG_160))
print("Train+Unsup Files: {}".format(len(train_in_valid_items)))

All Files: 26348
Train Files: 14669
Unsup Files: 7750
Valid Files: 3929

Train+Unsup Files: 22419
Train+Unsup Files: 1275


## Train with all data in `/train`, `/unsup` and `/val`

In [10]:
dblock = DataBlock(blocks=(ImageBlock(cls=PILImageInput), ImageBlock),
                   splitter=RandomSplitter(valid_pct=0),
                   get_items=get_all_items, 
                   get_y=lambda o: o)

dbunch =  dblock.databunch(source, path=source, bs=bs, num_workers=workers, 
                        item_tfms=item_tfms, batch_tfms=batch_tfms)

#CHANGE: We're predicting pixel values, so we're just going to predict an output for each RGB channel
dbunch.vocab = ['R', 'G', 'B']

print("Training Size:", len(dbunch.train_ds))
print("Validation Size:", len(dbunch.valid_ds))

Training Size: 26348
Validation Size: 0


In [ ]:
learn = unet_learner(dbunch, m, opt_func=opt_func, metrics=[], loss_func=MSELoss())
if dump: print(learn.model); exit()
if fp16: learn = learn.to_fp16()
cbs = MixUp(mixup) if mixup else []
learn.fit_flat_cos(epochs, lr, wd=1e-2, cbs=cbs)

# I'm not using fastai2's .export() because I only want to save 
# the model's parameters. 
torch.save(learn.model[0].state_dict(), 'all_train_unsup_val_pretext.pth')

epoch,train_loss,valid_loss,time


## Train with all data in `/train` and `/unsup`

In [ ]:

dblock = DataBlock(blocks=(ImageBlock(cls=PILImageInput), ImageBlock),
                   splitter=RandomSplitter(valid_pct=0),
                   get_items=get_train_and_unsup, 
                   get_y=lambda o: o)
dbunch =  dblock.databunch(source, path=source, bs=bs, num_workers=workers, 
                        item_tfms=item_tfms, batch_tfms=batch_tfms)

#CHANGE: We're predicting pixel values, so we're just going to predict an output for each RGB channel
dbunch.vocab = ['R', 'G', 'B']

print("Training Size:", len(dbunch.train_ds))
print("Validation Size:", len(dbunch.valid_ds))

In [ ]:
learn = unet_learner(dbunch, m, opt_func=opt_func, metrics=[], loss_func=MSELoss())
if dump: print(learn.model); exit()
if fp16: learn = learn.to_fp16()
cbs = MixUp(mixup) if mixup else []
learn.fit_flat_cos(epochs, lr, wd=1e-2, cbs=cbs)

# I'm not using fastai2's .export() because I only want to save 
# the model's parameters. 
torch.save(learn.model[0].state_dict(), 'all_train_unsup_pretext.pth')

## Train with all data in `/train`

In [ ]:
dblock = DataBlock(blocks=(ImageBlock(cls=PILImageInput), ImageBlock),
                   splitter=RandomSplitter(valid_pct=0),
                   get_items=get_train_items, 
                   get_y=lambda o: o)

dbunch =  dblock.databunch(source, path=source, bs=bs, num_workers=workers, 
                        item_tfms=item_tfms, batch_tfms=batch_tfms)

#CHANGE: We're predicting pixel values, so we're just going to predict an output for each RGB channel
dbunch.vocab = ['R', 'G', 'B']

print("Training Size:", len(dbunch.train_ds))
print("Validation Size:", len(dbunch.valid_ds))

In [ ]:
learn = unet_learner(dbunch, m, opt_func=opt_func, metrics=[], loss_func=MSELoss())
if dump: print(learn.model); exit()
if fp16: learn = learn.to_fp16()
cbs = MixUp(mixup) if mixup else []
learn.fit_flat_cos(epochs, lr, wd=1e-2, cbs=cbs)

# I'm not using fastai2's .export() because I only want to save 
# the model's parameters. 
torch.save(learn.model[0].state_dict(), 'all_train_pretext.pth')

## Train with partial data from `/train`

In [ ]:
dblock = DataBlock(blocks=(ImageBlock(cls=PILImageInput), ImageBlock),
                   splitter=RandomSplitter(valid_pct=0),
                   get_items=get_train_items_that_are_present_in_val, 
                   get_y=lambda o: o)

dbunch =  dblock.databunch(source, path=source, bs=bs, num_workers=workers, 
                        item_tfms=item_tfms, batch_tfms=batch_tfms)

#CHANGE: We're predicting pixel values, so we're just going to predict an output for each RGB channel
dbunch.vocab = ['R', 'G', 'B']

print("Training Size:", len(dbunch.train_ds))
print("Validation Size:", len(dbunch.valid_ds))

In [ ]:
learn = unet_learner(dbunch, m, opt_func=opt_func, metrics=[], loss_func=MSELoss())
if dump: print(learn.model); exit()
if fp16: learn = learn.to_fp16()
cbs = MixUp(mixup) if mixup else []
learn.fit_flat_cos(epochs, lr, wd=1e-2, cbs=cbs)

# I'm not using fastai2's .export() because I only want to save 
# the model's parameters. 
torch.save(learn.model[0].state_dict(), 'partial_train_pretext.pth')

# Downstream Task: Image网

Now that we've trained models on our pretext tasks, let's compare the performance of each model against one another.

In [ ]:
def get_dbunch(size, bs, sh=0., workers=None):
    if size<=224: 
        path = URLs.IMAGEWANG_160
    else: 
        path = URLs.IMAGEWANG
    source = untar_data(path)
    if workers is None: workers = min(8, num_cpus())
    dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                       splitter=GrandparentSplitter(valid_name='val'),
                       get_items=get_image_files, get_y=parent_label)
    item_tfms=[RandomResizedCrop(size, min_scale=0.35), FlipItem(0.5)]
    batch_tfms=RandomErasing(p=0.9, max_count=3, sh=sh) if sh else None
    return dblock.databunch(source, path=source, bs=bs, num_workers=workers,
                            item_tfms=item_tfms, batch_tfms=batch_tfms)

## All data in `/train`, `/unsup` and `/val`

In [ ]:
for run in range(runs):
        print(f'Run: {run}')
        #CHANGE: No self-attention
        sa = 0
        learn = Learner(dbunch, m(c_out=20, act_cls=torch.nn.ReLU, sa=sa, sym=sym, pool=pool), opt_func=opt_func, \
                metrics=[accuracy,top_k_accuracy], loss_func=LabelSmoothingCrossEntropy())
        if dump: print(learn.model); exit()
        if fp16: learn = learn.to_fp16()
        cbs = MixUp(mixup) if mixup else []
            
        # Load weights generated from training on our pretext task
        state_dict = torch.load('all_train_unsup_val_pretext.pth')
        # HACK: If we don't have all of the parameters for our learner, we get an error
        linear_layer = learn.model[-1]
        state_dict['11.weight'] = linear_layer.weight
        state_dict['11.bias'] = linear_layer.bias
        
        learn.model.load_state_dict(state_dict)
        
        learn.freeze()
        learn.fit_flat_cos(epochs, lr, wd=1e-2, cbs=cbs)

## All data in `/train` and `/unsup`

In [ ]:
for run in range(runs):
        print(f'Run: {run}')
        #CHANGE: No self-attention
        sa = 0
        learn = Learner(dbunch, m(c_out=20, act_cls=torch.nn.ReLU, sa=sa, sym=sym, pool=pool), opt_func=opt_func, \
                metrics=[accuracy,top_k_accuracy], loss_func=LabelSmoothingCrossEntropy())
        if dump: print(learn.model); exit()
        if fp16: learn = learn.to_fp16()
        cbs = MixUp(mixup) if mixup else []
            
        # Load weights generated from training on our pretext task
        state_dict = torch.load('all_train_unsup_pretext.pth')
        # HACK: If we don't have all of the parameters for our learner, we get an error
        linear_layer = learn.model[-1]
        state_dict['11.weight'] = linear_layer.weight
        state_dict['11.bias'] = linear_layer.bias
        
        learn.model.load_state_dict(state_dict)
        
        learn.freeze()
        learn.fit_flat_cos(epochs, lr, wd=1e-2, cbs=cbs)

## All data in `/train`

In [ ]:
for run in range(runs):
        print(f'Run: {run}')
        #CHANGE: No self-attention
        sa = 0
        learn = Learner(dbunch, m(c_out=20, act_cls=torch.nn.ReLU, sa=sa, sym=sym, pool=pool), opt_func=opt_func, \
                metrics=[accuracy,top_k_accuracy], loss_func=LabelSmoothingCrossEntropy())
        if dump: print(learn.model); exit()
        if fp16: learn = learn.to_fp16()
        cbs = MixUp(mixup) if mixup else []
            
        # Load weights generated from training on our pretext task
        state_dict = torch.load('all_train_pretext.pth')
        # HACK: If we don't have all of the parameters for our learner, we get an error
        linear_layer = learn.model[-1]
        state_dict['11.weight'] = linear_layer.weight
        state_dict['11.bias'] = linear_layer.bias
        
        learn.model.load_state_dict(state_dict)
        
        learn.freeze()
        learn.fit_flat_cos(epochs, lr, wd=1e-2, cbs=cbs)


## Partial data from `/train`

In [ ]:
for run in range(runs):
        print(f'Run: {run}')
        #CHANGE: No self-attention
        sa = 0
        learn = Learner(dbunch, m(c_out=20, act_cls=torch.nn.ReLU, sa=sa, sym=sym, pool=pool), opt_func=opt_func, \
                metrics=[accuracy,top_k_accuracy], loss_func=LabelSmoothingCrossEntropy())
        if dump: print(learn.model); exit()
        if fp16: learn = learn.to_fp16()
        cbs = MixUp(mixup) if mixup else []
            
        # Load weights generated from training on our pretext task
        state_dict = torch.load('partial_train_pretext.pth')
        # HACK: If we don't have all of the parameters for our learner, we get an error
        linear_layer = learn.model[-1]
        state_dict['11.weight'] = linear_layer.weight
        state_dict['11.bias'] = linear_layer.bias
        
        learn.model.load_state_dict(state_dict)
        
        learn.freeze()
        learn.fit_flat_cos(epochs, lr, wd=1e-2, cbs=cbs)
